In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import List, NamedTuple, Optional
import scipy.sparse as sp
# from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from tqdm import tqdm

import sys
sys.path.append('../')
import parser
import dataset

import datetime
from datetime import timedelta

from custom_parser import get_parser

import numpy as np 
import pandas as pd 
import torch
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from torch_geometric.utils import from_networkx, to_undirected
from torch_geometric.data import Data, DataLoader, Dataset, NeighborSampler
from torch_geometric.nn import SAGEConv

from torch_cluster import random_walk

from torch import Tensor
# from torch_geometric.data import InMemoryDataset

from tqdm import tqdm, tqdm_notebook, trange
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression

from collections import defaultdict
import random
from xgboost import XGBClassifier
%config Completer.use_jedi = False


In [8]:
# torch.cuda.get_device_name(0)

In [9]:
# data = dataset.Tdata(path='../../tdata.csv')
# parser = get_parser()
# args = parser.parse_args(args=
#                          ["--data","real-t", 
#                           "--sampling","xgb",
#                           "--mode","scratch",
#                           "--train_from","20140101",
#                           "--test_from","20170101",
#                           "--test_length","365",
#                           "--valid_length","180",
#                           "--initial_inspection_rate", "3",
#                           "--final_inspection_rate", "10",
#                          ])

In [11]:
# # args
# seed = args.seed
# epochs = args.epoch
# dim = args.dim
# lr = args.lr
# weight_decay = args.l2
# initial_inspection_rate = args.initial_inspection_rate
# inspection_rate_option = args.inspection_plan
# mode = args.mode
# train_begin = args.train_from 
# test_begin = args.test_from
# test_length = args.test_length
# valid_length = args.valid_length
# chosen_data = args.data
# numWeeks = args.numweeks
# semi_supervised = args.semi_supervised
# save = args.save
# gpu_id = args.device

# # Initial dataset split
# np.random.seed(seed)
# torch.manual_seed(seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(seed)

# # Initial dataset split
# train_start_day = datetime.date(int(train_begin[:4]), int(train_begin[4:6]), int(train_begin[6:8]))
# test_start_day = datetime.date(int(test_begin[:4]), int(test_begin[4:6]), int(test_begin[6:8]))
# test_length = timedelta(days=test_length)    
# test_end_day = test_start_day + test_length
# valid_length = timedelta(days=valid_length)
# valid_start_day = test_start_day - valid_length

# # data
# data.split(train_start_day, valid_start_day, test_start_day, test_end_day, valid_length, test_length, args)
# data.featureEngineering()

Data size:
Train labeled: (40475, 41), Train unlabeled: (1308679, 41), Valid labeled: (437124, 41), Valid unlabeled: (0, 13), Test: (858180, 41)
Checking label distribution
Training: 0.07383529661466624
Validation: 0.07495764097746672
Testing: 0.0957648251549135


In [12]:
# sys.path.append('../graph_sage')
# from utils import *
# from pygData_util import *

In [13]:
# categories=["importer.id","HS6"]
# gdata = GraphData(data,use_xgb=True, categories=categories)

Training XGBoost model...


In [14]:
# best_thresh, best_auc = find_best_threshold(gdata.xgb,data.dfvalidx_lab, data.valid_cls_label)
# xgb_test_pred = gdata.xgb.predict_proba(data.dfvalidx_lab)[:,-1]
# overall_f1,auc,pr, re, f, rev = metrics(xgb_test_pred, data.valid_cls_label,data.valid_reg_label,best_thresh)
# print("-"*50)
# xgb_test_pred = gdata.xgb.predict_proba(data.dftestx)[:,-1]
# overall_f1,auc,pr, re, f, rev = metrics(xgb_test_pred, data.test_cls_label,data.test_reg_label,best_thresh)

Checking top 1% suspicious transactions: 4371
Precision: 0.6287, Recall: 0.0902, Revenue: 0.1119
Checking top 2% suspicious transactions: 8742
Precision: 0.5420, Recall: 0.1554, Revenue: 0.1860
Checking top 5% suspicious transactions: 21857
Precision: 0.3742, Recall: 0.2683, Revenue: 0.3006
Checking top 10% suspicious transactions: 43712
Precision: 0.2683, Recall: 0.3848, Revenue: 0.4311
--------------------------------------------------
Checking top 1% suspicious transactions: 8581
Precision: 0.5809, Recall: 0.0665, Revenue: 0.0778
Checking top 2% suspicious transactions: 17164
Precision: 0.4996, Recall: 0.1143, Revenue: 0.1364
Checking top 5% suspicious transactions: 42909
Precision: 0.3674, Recall: 0.2102, Revenue: 0.2498
Checking top 10% suspicious transactions: 85818
Precision: 0.2769, Recall: 0.3168, Revenue: 0.3690


In [15]:
# stage = "train_lab"
# trainLab_data = gdata.get_data(stage)
# train_nodeidx = torch.tensor(gdata.get_AttNode(stage))
# trainLab_data.node_idx = train_nodeidx

In [16]:
# stage = "train_unlab"
# unlab_data = gdata.get_data(stage)
# unlab_nodeidx = torch.tensor(gdata.get_AttNode(stage))
# unlab_data.node_idx = unlab_nodeidx

In [17]:
# stage = "valid"
# valid_data = gdata.get_data(stage)
# valid_nodeidx = torch.tensor(gdata.get_AttNode(stage))
# valid_data.node_idx = valid_nodeidx

In [18]:
# torch.save(valid_data, 'valid_data.pt')

In [13]:
# trainLab_data, gdata.leaf_dim
# gdata.leaf_dim # 1549

In [19]:
train_lab_data = torch.load('train_data.pt')
train_lab_data

valid_lab_data = torch.load('valid_data.pt')
valid_lab_data

Data(edge_attr=[874248], edge_index=[2, 1748496], edge_label=[1748496], node_idx=[437124], rev=[467973], x=[467973, 100], y=[467973])

# Data

In [56]:
class Batch(NamedTuple):
    '''
    convert batch data for pytorch-lightning
    '''
    x: Tensor
    y: Tensor
    rev: Tensor
    adjs_t: NamedTuple

    def to(self, *args, **kwargs):
        return Batch(
            x=self.x.to(*args, **kwargs),
            y=self.y.to(*args, **kwargs),
            rev=self.rev.to(*args, **kwargs),
            adjs_t=[(adj_t.to(*args, **kwargs), eid.to(*args, **kwargs), size) for adj_t, eid, size in self.adjs_t],
        )
    
class UnsupNeighborSampler(NeighborSampler):
    
    def sample(self, batch):
        batch = torch.tensor(batch)
        row, col, _ = self.adj_t.coo()

        # For each node in `batch`, we sample a direct neighbor (as positive
        # example) and a random node (as negative example):
        pos_batch = random_walk(row, col, batch, walk_length=1,
                                coalesced=False)[:, 1]

        neg_batch = torch.randint(0, self.adj_t.size(1), (batch.numel(), ),
                                  dtype=torch.long)

        batch = torch.cat([batch, pos_batch, neg_batch], dim=0)
        return super(UnsupNeighborSampler, self).sample(batch)
    
class CustomData(LightningDataModule):
    def __init__(self,train_data, valid_data, sizes, batch_size):
        super(CustomData,self).__init__()

        self.train_data = train_data
        self.valid_data = valid_data
        
        self.sizes = sizes
        self.batch_size = batch_size

    def train_dataloader(self):
        return UnsupNeighborSampler(self.train_data.edge_index, node_idx=self.train_data.node_idx,
                               sizes=self.sizes, return_e_id=True,
                               batch_size=self.batch_size,
                               shuffle=True,
                               drop_last=True,
                               transform=self.convert_train_batch,
                               num_workers=16)

    def val_dataloader(self):
        return NeighborSampler(self.valid_data.edge_index, 
                               node_idx=self.valid_data.node_idx,
                               sizes=self.sizes, 
                               return_e_id=True,
                               batch_size=self.batch_size,
                               shuffle=False,
                               drop_last=True,
                               transform=self.convert_valid_batch
                              )

    def convert_train_batch(self, batch_size, n_id, adjs):
        return Batch(
            x=self.train_data.x[n_id],
            y=self.train_data.y[n_id[:batch_size]],
            rev = self.train_data.rev[n_id[:batch_size]],
            adjs_t=adjs,
        )
    
    
    def convert_valid_batch(self, batch_size, n_id, adjs):
        return Batch(
            x=self.valid_data.x[n_id],
            y=self.valid_data.y[n_id[:batch_size]],
            rev = self.valid_data.rev[n_id[:batch_size]],
            adjs_t=adjs,
        )

# Model

In [57]:
class LabelPredictor(nn.Module):
    def __init__(self,in_channels):
        super(LabelPredictor, self).__init__()
        
        self.linear = nn.Linear(in_channels * 3, 1)
        
    def forward(self, x): 
      
        emb_a = x
        emb_b = x.roll(1,0)
        
        print(emb_b)
        print(emb_a)
        emb_abs = torch.abs(emb_a - emb_b)
        emb_sum = emb_a + emb_b
        emb_mult = emb_a * emb_b
        
        x = torch.cat([emb_abs, emb_sum, emb_mult], dim=-1)

        x = self.linear(x)
        x = torch.sigmoid(x)
        
        return x
    
lp = LabelPredictor(4)

x = torch.rand(2,4)
print(x)
lp(x).shape
 

tensor([[0.7006, 0.9475, 0.2457, 0.5078],
        [0.2457, 0.0703, 0.7142, 0.6278]])
tensor([[0.2457, 0.0703, 0.7142, 0.6278],
        [0.7006, 0.9475, 0.2457, 0.5078]])
tensor([[0.7006, 0.9475, 0.2457, 0.5078],
        [0.2457, 0.0703, 0.7142, 0.6278]])


torch.Size([2, 1])

In [58]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers, leaf_len):
        super(SAGE, self).__init__()
        
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        self.emb = nn.Embedding(leaf_len, in_channels, padding_idx=0)
        self.bn = nn.BatchNorm1d(in_channels)
            
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv(in_channels, hidden_channels))

        
    def forward(self, x, adjs):
        
        x = self.emb(x)

        x = torch.sum(x,dim=1) # summation over the leaves
        x = F.relu(self.bn(x))
        
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
                
        return x

#     def full_forward(self, x, edge_index):
        
#         x = self.emb(x)

#         x = torch.sum(x,dim=1) # summation over the leaves
#         x = F.relu(self.bn(x))
        
#         for i, conv in enumerate(self.convs):
#             x = conv(x, edge_index)
#             if i != self.num_layers - 1:
#                 x = x.relu()
#                 x = F.dropout(x, p=0.5, training=self.training)
#         return x

In [59]:
class SageModel(pl.LightningModule):
    def __init__(self, X, Y, **kwargs):
        super(SageModel, self).__init__()
        
        self.sage = SAGE(**kwargs)

        self.X = None
        self.y = None

    def forward(self, x, adjs):
        
        return self.sage(x, adjs)
        
    
    def full_forward(self, x, adjs):

        return self.sage.full_forward(x, adjs)
        
    def training_step(self, batch, batch_idx):
        
        x, y, rev, adjs = batch
        
        out = self.forward(x, adjs)
        
        out, pos_out, neg_out = out.split(out.size(0) // 3, dim=0)

        pos_loss = F.logsigmoid((out * pos_out).sum(-1)).mean()
        neg_loss = F.logsigmoid(-(out * neg_out).sum(-1)).mean()
        loss = -pos_loss - neg_loss
        
        if not self.X is None:
            self.X = torch.cat([self.X, out])
            self.y = torch.cat([self.y, y.flatten()])
        else:
            self.X = out
            self.y = y.flatten()
        
        return loss

    def validation_step(self, batch, batch_idx):
        
        x, y, rev, adjs = batch
        
        features = self.forward(x, adjs)
                
        return {'features':features, 'y': y.flatten()}
        
    def validation_epoch_end(self, outputs):
        
        valid_X = torch.cat([x["features"] for x in outputs], dim=0)
        valid_y = torch.cat([x["y"] for x in outputs], dim=0)
        
        print(self.y.shape)
        
        clf = LogisticRegression()
        clf.fit(self.X, self.y)
        
        score = clf.score(valid_X, valid_y)
        
        self.log('val_acc', score, prog_bar=True)
            
        self.X = None
        self.y = None
            
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [61]:
leaf_len = 1549

batch_size = 256
sizes = [-1, 200]

data_loader = CustomData(train_lab_data, valid_lab_data, sizes, batch_size)

model = SageModel(
    train_lab_data.x, 
    train_lab_data.y, 
    in_channels=128, 
    hidden_channels=64, 
    num_layers=2, 
    leaf_len=leaf_len)

trainer = Trainer(num_sanity_val_steps=0)
trainer.fit(model, data_loader)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name | Type | Params
------------------------------
0 | sage | SAGE | 223 K 
------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params


Epoch 0:   1%|          | 17/1865 [01:25<2:35:09,  5.04s/it, loss=1.95, v_num=36]


1